## Imports

In [ ]:
%cd ~/Documents/PHD/repos/RL_audio/notebooks

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

%matplotlib widget
# %matplotlib inline
# %matplotlib notebook

# Try to use this website to use the explode feature, so we can see internal blocks and space everything out
# https://terbium.io/2017/12/matplotlib-3d/ 

## Initializations

In [ ]:
user_ID_str = "00"
load_file = "lastsave"

In [ ]:
st0_array = np.load("user_data/user_" + user_ID_str + "/arrays/" + load_file + "_st0.npy")
st1_array = np.load("user_data/user_" + user_ID_str + "/arrays/" + load_file + "_st1.npy")
st2_array = np.load("user_data/user_" + user_ID_str + "/arrays/" + load_file + "_st2.npy")

## Visualization of Arrays

### Print Arrays

In [ ]:
print(st0_array.shape,"\n")
print(st0_array)

In [ ]:
print(st1_array.shape,"\n")
print(st1_array)

In [ ]:
print(st2_array.shape,"\n")

print(st2_array)

### Plotter

In [ ]:
# STATE 0

fig = plt.figure(figsize=(9, 9))
ax = fig.add_subplot(projection='3d')
ax.set_title("STATE 0 - PROGRESSING", size=15)

cmap = plt.get_cmap("RdYlGn")
norm= plt.Normalize(st0_array.min(), st0_array.max())

filled = np.ones_like(st0_array)

# Print the voxel cube
cube = ax.voxels(filled, facecolors=cmap(norm(st0_array)), edgecolor="white", alpha=0.75)

# To get the colour bar (note scatter size is set to zero
counter = range(3)
x,y,z = np.meshgrid(counter, counter, counter)
scatter_cube = ax.scatter(x,y,z, c=st0_array.flat, cmap=plt.get_cmap("RdYlGn"), s=0, alpha=0.8, linewidths=2, edgecolors="black", vmin=-10, vmax=10)
cbar = fig.colorbar(scatter_cube, shrink=0.75, orientation="horizontal", aspect=15, pad=0.05) # Add a color bar
cbar.minorticks_on()

ax.set_xticks([])
# for minor ticks
# ax.set_xticks([], minor=True)
ax.set_yticks([])
ax.set_zticks([])

plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=True)

ax.set_xlabel('0    -    1     -    2\nBeat Per Min', size=15)
ax.set_ylabel('0    -    1     -    2\nBeat Per Loop', size=15)
ax.set_zlabel('0    -    1     -    2\nPitch Bend', size=15)

plt.show()

In [ ]:
# STATE 1

fig = plt.figure(figsize=(9, 9))
ax = fig.add_subplot(projection='3d')
ax.set_title("STATE 1 - SUCCESSFUL", size=15)

cmap = plt.get_cmap("RdYlGn")
norm= plt.Normalize(st1_array.min(), st1_array.max())

filled = np.ones_like(st1_array)

# Print the voxel cube
cube = ax.voxels(filled, facecolors=cmap(norm(st1_array)), edgecolor="white", alpha=0.75)

# To get the colour bar (note scatter size is set to zero
counter = range(3)
x,y,z = np.meshgrid(counter, counter, counter)
scatter_cube = ax.scatter(x,y,z, c=st1_array.flat, cmap=plt.get_cmap("RdYlGn"), s=0, alpha=0.8, linewidths=2, edgecolors="black", vmin=-10, vmax=10)
cbar = fig.colorbar(scatter_cube, shrink=0.75, orientation="horizontal", aspect=15, pad=0.05) # Add a color bar
cbar.minorticks_on()

ax.set_xticks([])
# for minor ticks
# ax.set_xticks([], minor=True)
ax.set_yticks([])
ax.set_zticks([])

plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=True)

ax.set_xlabel('0    -    1     -    2\nBeat Per Min', size=15)
ax.set_ylabel('0    -    1     -    2\nBeat Per Loop', size=15)
ax.set_zlabel('0    -    1     -    2\nPitch Bend', size=15)

plt.show()

In [ ]:
# STATE 2

fig = plt.figure(figsize=(9, 9))
ax = fig.add_subplot(projection='3d')
ax.set_title("STATE 2 - STUCK", size=15)

cmap = plt.get_cmap("RdYlGn")
norm= plt.Normalize(st2_array.min(), st2_array.max())

filled = np.ones_like(st2_array)

# Print the voxel cube
cube = ax.voxels(filled, facecolors=cmap(norm(st2_array)), edgecolor="white", alpha=0.75)

# To get the colour bar (note scatter size is set to zero
counter = range(3)
x,y,z = np.meshgrid(counter, counter, counter)
scatter_cube = ax.scatter(x,y,z, c=st2_array.flat, cmap=plt.get_cmap("RdYlGn"), s=0, alpha=0.8, linewidths=2, edgecolors="black", vmin=-10, vmax=10)
cbar = fig.colorbar(scatter_cube, shrink=0.75, orientation="horizontal", aspect=15, pad=0.05) # Add a color bar
cbar.minorticks_on()

ax.set_xticks([])
# for minor ticks
# ax.set_xticks([], minor=True)
ax.set_yticks([])
ax.set_zticks([])

plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=True)

ax.set_xlabel('0    -    1     -    2\nBeat Per Min', size=15)
ax.set_ylabel('0    -    1     -    2\nBeat Per Loop', size=15)
ax.set_zlabel('0    -    1     -    2\nPitch Bend', size=15)

plt.show()

### ARCHIVE

In [ ]:
# OLD SCATTER

# import matplotlib.pyplot as plt
# import numpy as np

# from mpl_toolkits.mplot3d import Axes3D
# M = st0_array
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# counter = range(3)
# x,y,z = np.meshgrid(counter, counter, counter)

# scatter_cube = ax.scatter(x,y,z, c=M.flat, cmap=plt.get_cmap("RdYlGn"), s=100, alpha=0.8, linewidths=2, edgecolors="black", vmin=-10, vmax=10)
# cbar = fig.colorbar(scatter_cube, shrink=0.6, aspect=5) # Add a color bar

# plt.show()